# Metro Manila Film Festival | Movie and Awards Database

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display, HTML
import pickle

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

with open('all_movies.pkl', 'rb') as f:
    all_movies = pickle.load(f)
with open('awards_df.pkl', 'rb') as g:
    awards_df = pickle.load(g)

from IPython.display import display, HTML

def load_custom_css():
    styles = open("custom.css", "r").read()
    return HTML(f"<style>{styles}</style>")

load_custom_css()

In [2]:
filter_tab_content = widgets.Output()
with filter_tab_content:
    label = widgets.Label(value=
          'Can filter based on year, movie, actor, studio, director, or genre. You can also choose to input "all movies"')
    
    # CREATE FILTER BUTTON
    filter_input = widgets.Text(value='', description='Filter by:')
    filter_button = widgets.Button(description='Submit')
    filter_output = widgets.Output()
    
    # STARTING FILTER_DF
    filter_df = all_movies.copy()
    del filter_df['Link']
    
    # ASSIGN RESULTS ONCE `filter_button` IS CLICKED
    def on_filter_button_click(b):
        global filter_df, filter_options_list
        global year_input, movie_input, actor_input, studio_input, director_input, genre_input
        global sort_category_input, sort_order_input
    
        with filter_output:
            filter_output.clear_output()
            filter_options = filter_input.value
    
            # FILTER OPTIONS LIST
            valid_inputs_list = ['year', 'movie', 'actor', 'studio', 'director', 'genre', 'all movies', '']
            filter_options_list = filter_options.split(',')
            filter_options_list = [item.strip().lower() for item in filter_options_list]
    
            # CHECK VALIDITY OF FILTER VALUE
            validity = True
            for item in filter_options_list:
                if not(item in valid_inputs_list):
                    validity = False
                    break
    
            if validity == False:
                print('Cannot filter based on inputted category')
    
            # FILTER TABLE
            else:
                if filter_options == '':
                    display()
    
                else:
                    if 'all movies' in filter_options_list:
                        filter_df = all_movies.copy()
                        del filter_df['Link']
                        filter_df = filter_df.sort_values('Year').set_index(['Year','Title'])
            
                    else:
                        display(HTML('<h3>FILTER VALUES:</h3>'))
                        if 'year' in filter_options_list:
                            label = widgets.Label(value="Enter year/s:")
                            year_input = widgets.Text(value='')
                            display(label, year_input)
                
                        if 'movie' in filter_options_list:
                            label = widgets.Label(value="Enter movie title/s:")
                            movie_input = widgets.Text(value='')
                            display(label, movie_input)
            
                        if 'actor' in filter_options_list:
                            label = widgets.Label(value="Enter actor/s:")
                            actor_input = widgets.Text(value='')
                            display(label, actor_input)
            
                        if 'studio' in filter_options_list:
                            label = widgets.Label(value="Enter studio/s:")
                            studio_input = widgets.Text(value='')
                            display(label, studio_input)
            
                        if 'director' in filter_options_list:
                            label = widgets.Label(value="Enter director/s:")
                            director_input = widgets.Text(value='')
                            display(label, director_input)
            
                        if 'genre' in filter_options_list:
                            label = widgets.Label(value="Enter genre/s:")
                            genre_input = widgets.Text(value='')
                            display(label, genre_input)
    
    
                    # CREATE SORT INPUTS & VALUES BUTTON
                    label_cat = widgets.Label(value="Enter a category to sort the values by:")
                    sort_category_input = widgets.Dropdown(options=['Year','Movie Titles','IMDb Rating'], value='Year')
                
                    label_order = widgets.Label(value="Enter order of sorting:")
                    sort_order_input = widgets.Dropdown(options=['Ascending','Descending'], value='Ascending')
                
                    values_button = widgets.Button(description='Submit Filter Values')
                    values_button.on_click(on_values_button_click)
    
                    # DISPLAY SORT INPUTS & VALUES BUTTON
                    display(HTML('<h3>SORT VALUES:</h3>'))
                    display(label_cat, sort_category_input, label_order, sort_order_input)
                    display(values_button)
    
    # ASSIGN RESULTS ONCE `values_button` IS CLICKED
    def on_values_button_click(b):
        with filter_output:
            filter_output.clear_output()            
            filter_df = all_movies.copy()
            del filter_df['Link']
            year_valid = True
    
            # FILTER DF BASED ON INPUTTED VALUES
            if 'year' in filter_options_list:
                var = year_input.value
                filter_list = var.split(',')
                try:
                    filter_list = [int(item.strip()) for item in filter_list]
                    concat_list = [filter_df[filter_df['Year'] == item] for item in filter_list]
                    filter_df = pd.concat(concat_list, ignore_index=True)
                    print(f'Year Filter: {var}')
                except ValueError:
                    year_valid = False
    
            if 'movie' in filter_options_list:
                var = movie_input.value
                filter_list = var.split(',')
                filter_list = [item.strip() for item in filter_list]
                concat_list = [filter_df[filter_df['Title'].str.contains(item, case=False)] for item in filter_list]
                filter_df = pd.concat(concat_list, ignore_index=True)
                print(f'Movie Filter: {var}')
    
            if 'actor' in filter_options_list:
                var = actor_input.value
                filter_list = var.split(',')
                filter_list = [item.strip() for item in filter_list]
                concat_list = [filter_df[filter_df['Starring'].str.contains(item, case=False)] for item in filter_list]
                filter_df = pd.concat(concat_list, ignore_index=True)
                print(f'Actor Filter: {var}')
                
            if 'studio' in filter_options_list:
                var = studio_input.value
                filter_list = var.split(',')
                filter_list = [item.strip() for item in filter_list]
                concat_list = [filter_df[filter_df['Studio'].str.contains(item, case=False)] for item in filter_list]
                filter_df = pd.concat(concat_list, ignore_index=True)
                print(f'Studio Filter: {var}')
                
            if 'director' in filter_options_list:
                var = director_input.value
                filter_list = var.split(',')
                filter_list = [item.strip() for item in filter_list]
                concat_list = [filter_df[filter_df['Director'].str.contains(item, case=False)] for item in filter_list]
                filter_df = pd.concat(concat_list, ignore_index=True)
                print(f'Director Filter: {var}')
                
            if 'genre' in filter_options_list:
                var = genre_input.value
                filter_list = var.split(',')
                filter_list = [item.strip() for item in filter_list]
                for item in filter_list:
                    filter_df = filter_df[filter_df['Genre'].str.contains(item, case=False)]
                print(f'Genre Filter: {var}')
    
            filter_df = filter_df.set_index(['Year','Title'])
            if year_valid == False:
                print('Sorry, year input should be a number!')
            elif filter_df.shape[0] == 0:
                print('Sorry, no movies fit the inputted criteria!')
    
            # SORT DF BASED ON INPUTS
            else:
                cat = sort_category_input.value
                order = sort_order_input.value

                filter_df['IMDb Rating'] = filter_df['IMDb Rating'].apply(lambda x: -1 if x == 'No IMDb Rating' else x)
                
                if cat == 'Movie Titles':
                    cat = 'Title'
                    
                filter_df = filter_df.sort_values(cat, ascending= order=='Ascending')
    
                print(f'Sort Category: {cat}')
                print(f'Sort Order: {order}')

                filter_df['IMDb Rating'] = filter_df['IMDb Rating'].apply(lambda x: 'No IMDb Rating' if x == -1 else x)
                display(filter_df)
    
    # DISPLAY: FILTER INPUT, FILTER BUTTON, OUTPUT
    filter_button.on_click(on_filter_button_click)
    display(label, filter_input)
    display(filter_button, filter_output)

In [3]:
reviews_tab_content = widgets.Output()
with reviews_tab_content:
    label = HTML('<h3>Enter a category to show its corresponding awards!</h3>')
    
    # TEXTBOX AND BUTTON FOR MOVIE INPUT
    cat_rev_label = widgets.Label(value = 'Enter category:')
    cat_rev_input = widgets.Dropdown(options=['Year','Movie Title','Artist','Award'], value='Year')
    
    cat_rev_button = widgets.Button(description='Submit')
    rev_output = widgets.Output()
    
    # ASSIGN VARIABLES
    awards_filter_df = awards_df.copy()
    
    # DEFINE OUTPUT ONCE BUTTON IS CLICKED
    def on_cat_button_click(b):
        global value_awards_input, category
        with rev_output:
            rev_output.clear_output()
            category = cat_rev_input.value
    
            if category == '':
                display()
            else:
                label = widgets.Label(value=f"Enter {category}:")
                value_awards_input = widgets.Text(value='')
                display(label, value_awards_input)
            
            value_rev_button = widgets.Button(description='Submit')
            value_rev_button.on_click(on_value_button_click)
            display(value_rev_button)
    
    
    #########
    
    def on_value_button_click(b):
        global awards_filter_df
        with rev_output:
            rev_output.clear_output()
            var = value_awards_input.value
            awards_filter_df = awards_df.copy()
            
            if category == 'Year':
                try:
                    int(var)
                    print(f'Year Filter: {var}')
                    awards_filter_df = awards_filter_df[awards_filter_df['Year'] == int(var)]
                    
                    for index, row in awards_filter_df.iterrows():
                        award = row.iloc[1]
                        movie_winner = row.iloc[2]
                        artist_winner = row.iloc[3]
                        
                        display(HTML(f'<h1>{award}</h1>'))
                        if artist_winner != '—':
                            display(HTML(f'<b>{artist_winner}</b>'))
                        display(HTML(f'<b>{movie_winner}</b>'))
                        display(HTML('<hr>'))
                        
                except ValueError:
                    print('Sorry, year input should be a number!')
                    
        
            elif category == 'Movie Title':
                print(f'Movie Filter: {var}')
                awards_filter_df = awards_filter_df[awards_filter_df['Movie Winner'].str.contains(var, case=False)]

                if awards_filter_df.shape[0] == 0:
                    print(f'"{var}" has not won any award or was not in MMFF')
                else:
                    for index, row in awards_filter_df.iterrows():
                        year = row.iloc[0]
                        award = row.iloc[1]
                        movie_winner = row.iloc[2]
                        artist_winner = row.iloc[3]

                        display(HTML(f'<h1>{award}</h1>'))
                        if artist_winner != '—':
                            display(HTML(f'<b>{artist_winner}</b>'))
                        display(HTML(f'<b>{movie_winner}</b>'))
                        display(HTML('<hr>'))

            elif category == 'Artist':
                print(f'Artist Filter: {var}')
                awards_filter_df = awards_filter_df[awards_filter_df['Artist Winner'].str.contains(var, case=False)]

                if awards_filter_df.shape[0] == 0:
                    print(f'{var} has not won any award in MMFF')
                else:
                    for index, row in awards_filter_df.iterrows():
                        year = row.iloc[0]
                        award = row.iloc[1]
                        movie_winner = row.iloc[2]
                        artist_winner = row.iloc[3]
    
                        display(HTML(f'<h1>{award}</h1>'))
                        if artist_winner != '—':
                            display(HTML(f'<b>{artist_winner}</b>'))
                        display(HTML(f'<b>{movie_winner}</b>'))
                        display(HTML('<hr>'))

            elif category == 'Award':
                print(f'Artist Filter: {var}')
                awards_filter_df = awards_filter_df[awards_filter_df['Award'].str.contains(var, case=False)]

                if awards_filter_df.shape[0] == 0:
                    print(f'{var} is not given in MMFF')
                else:
                    for index, row in awards_filter_df.iterrows():
                        year = row.iloc[0]
                        award = row.iloc[1]
                        movie_winner = row.iloc[2]
                        artist_winner = row.iloc[3]
    
                        display(HTML(f'<h1>{year} {award}</h1>'))
                        if artist_winner != '—':
                            display(HTML(f'<b>{artist_winner}</b>'))
                        display(HTML(f'<b>{movie_winner}</b>'))
                        display(HTML('<hr>'))
        
    cat_rev_button.on_click(on_cat_button_click)
    display(label, cat_rev_label, cat_rev_input)
    display(cat_rev_button, rev_output)

In [4]:
tabs = widgets.Tab()
tabs.children = [filter_tab_content, reviews_tab_content]
tabs.set_title(0, 'Filter Movies')
tabs.set_title(1, 'Movie Awards')

display(tabs)